<a href="https://colab.research.google.com/github/Ginko2501/SURP2022/blob/main/ATF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
import math
import numpy as np

In [3]:
class node (object):
  # a node contains a vertex, the nodal_ray attached to that vertex, and the edge departing from that vertex
  def __init__ (self, vertex, nodal_ray, edge, affine_length):
    self.vertex = vertex
    self.nodal_ray = nodal_ray
    self.edge = edge
    self.affine_length = affine_length

In [20]:
n = 0
nodes = np.array([])

In [18]:
def init_polydisk (x):
  # initialize polydisk P(1,x)
  global n
  global nodes

  n = 4
  nodes = np.array([])
  nodes = np.append(nodes, node(np.array([0,0]), np.array([1,1]), np.array([0,1]), 1))
  nodes = np.append(nodes, node(np.array([0,1]), np.array([1,-1]), np.array([1,0]), x))
  nodes = np.append(nodes, node(np.array([x,1]), np.array([-1,-1]), np.array([0,-1]), 1))
  nodes = np.append(nodes, node(np.array([x,0]), np.array([-1,1]), np.array([-1,0]), x))

In [22]:
# test init_polydisk
init_polydisk(2)

for i in range(n):
  print(nodes[i].vertex, end=" ")
print()

for i in range(n):
  print(nodes[i].affine_length * nodes[i].edge, end=" ")

[0 0] [0 1] [2 1] [2 0] 
[0 1] [2 0] [ 0 -1] [-2  0] 

In [30]:
def dist (x,y):
  return np.linalg.norm(x-y)

In [47]:
def intersect_one (i,j):
  # solve the intersection between i-th nodal ray and j-th edge
  global n
  global nodes

  # copy as local variables
  n1 = nodes[i].vertex
  n2 = nodes[j].vertex
  n3 = nodes[(j+1)%n].vertex
  v1 = nodes[i].nodal_ray
  v2 = nodes[j].edge

  # solve for the intersection point
  vec = np.array([ v1[1]*n1[0]-v1[0]*n1[1], v2[1]*n2[0]-v2[0]*n2[1] ])
  mat = np.array([ [ v1[1], -v1[0] ],
            [ v2[1], -v2[0] ] ])
  itx = np.linalg.solve(mat, vec)

  # check the intersection is on the edge
  if (n2[0]==n3[0]):
    lmbda = (itx[1]-n3[1]) / (n2[1]-n3[1])
  else:
    lmbda = (itx[0]-n3[0]) / (n2[0]-n3[0])
  if (lmbda<0 or lmbda>1):
    return np.array([-1,-1])

  return itx

In [25]:
# test intersect_one
init_polydisk(0.5)
intersect_one(0,1)

-1

In [53]:
def intersect_all (x):
  # solve the intersecting edge by the x-th nodal ray
  global n
  global nodes

  # the variables for the intersecting edge
  min_edge = x
  min_itx = np.array([])
  min_dist = math.inf
  
  for i in range(n):
    if (i==x or i==(x-1)%n):
      continue

    itx = intersect_one(x,i)
    if (np.array_equal(itx, [-1,-1])):
      continue
    
    dis = dist(nodes[x].vertex, itx)
    if (dis < min_dist):
      min_edge = i
      min_itx = itx
      min_dist = dis

  return (min_edge, min_itx)

In [52]:
# test intersect_all
init_polydisk(0.5)
intersect_all(0)

(2, array([0.5, 0.5]))

In [21]:
def solve_matrix (v1, v2, w1, w2):
  # solve the eigen-direction
  vec = np.concatenate((v2,w2))
  mat = np.array([ [v1[0], v1[1], 0,   0  ],
            [0,   0,   v1[0], v1[1]], 
            [w1[0], w1[1], 0,  0,  ],
            [0,   0,   w1[0], w1[1]], ])
  res = np.linalg.solve(mat,vec)
  return np.array([ [res[0],res[1]], 
            [res[2],res[3]] ])

In [ ]:
# test solve_matrix
v1 = [-1,1]
v2 = [-1,1]
w1 = [0,1]
w2 = [1,0]
solve_matrix(v1,v2,w1,w2)

array([[ 2.,  1.],
       [-1.,  0.]])

In [ ]:
def mutate_clockwise (l, r):
  # mutate with nodal_ray < intersecting edge
  global n
  global nodes
  global nodal_rays
  global edges

  mat = solve_matrix(l,l, edges[l], edges[(l-1)%n])

In [ ]:
def mutate_counterclockwise (x):
  # mutate with nodal_ray > intersecting edge
  return x

In [ ]:
def mutate (x):
  # mutate once by x-th nodal_ray
  global n
  global nodes
  global nodal_rays
  global edges

  (y, itx) = intersect_all(x)

  if (x<y):
    return mutate_clockwise(x,y)
  else:
    return mutate_counterclockwise(x,y)